# Cleaning Recipe Data
Mainly, extract the recipe time from the instructions text and convert to minutes.  
Data Source: https://www.kaggle.com/datasets/elisaxxygao/foodrecsysv1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
datapath = '/content/drive/MyDrive/Colab Notebooks/Recipe ML Project/recipe_data/'

Mounted at /content/drive


In [ ]:
import pandas as pd
import json

Not sure why there are two files in the dataset... but both have the info we need, so we'll use just the 'raw' one.

In [ ]:
# core_recipes_df = pd.read_csv(datapath+'core-data_recipe.csv')
# core_recipes_df = core_recipes_df.drop(columns=['image_url', 'nutritions']) # leave out unnecessary columns
# core_recipes_df

In [ ]:
raw_recipes_df = pd.read_csv(datapath+'raw-data_recipe.csv', dtype={'cooking_directions':object})
raw_recipes_df = raw_recipes_df.drop(columns=['image_url', 'nutritions', 'reviews']) # leave out unnecessary columns
raw_recipes_df

,recipe_id,recipe_name,aver_rate,review_nums,ingredients,cooking_directions
0,222388,Homemade Bacon,5.000000,3,pork belly^smoked paprika^kosher salt^ground b...,{'directions': u'Prep\n5 m\nCook\n2 h 45 m\nRe...
1,240488,"Pork Loin, Apples, and Sauerkraut",4.764706,29,sauerkraut drained^Granny Smith apples sliced^...,{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...
2,218939,Foolproof Rosemary Chicken Wings,4.571429,12,chicken wings^sprigs rosemary^head garlic^oliv...,"{'directions': u""Prep\n20 m\nCook\n40 m\nReady..."
3,87211,Chicken Pesto Paninis,4.625000,163,focaccia bread quartered^prepared basil pesto^...,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...
4,245714,Potato Bacon Pizza,4.500000,2,red potatoes^strips bacon^Sauce:^heavy whippin...,{'directions': u'Prep\n20 m\nCook\n45 m\nReady...
...,...,...,...,...,...,...
49693,222886,Grateful Dead Cocktail,3.500000,4,fluid ounce tequila^fluid ounce vodka^fluid ou...,{'directions': u'Prep\n5 m\nReady In\n5 m\nCom...
49694,25650,Cheese Filling For Pastries,4.333333,3,raisins^brandy^cream cheese^white sugar^all-pu...,{'directions': u'Prep\n15 m\nReady In\n15 m\nS...
49695,23544,Peach Smoothie,3.615385,21,sliced peaches drained^scoops vanilla ice crea...,{'directions': u'Prep\n1 m\nReady In\n1 m\nIn ...
49696,170710,Double Dare Peaches,4.714286,19,butter^habanero peppers^fresh peaches^brown su...,{'directions': u'Prep\n20 m\nCook\n10 m\nReady...


# Extract the recipe time in minutes
The recipe time is in the first few lines of the cooking instructions, so we need to parse it out and make a new column.

In [ ]:
from zmq.constants import NULL
import ast
import re

# time_regex = re.compile(r"Prep\n.*\nCook\n.*\n(Ready In\n.*\n)?", re.MULTILINE)
readyin_regex = re.compile(r"Ready In\n.*\n", re.MULTILINE)

def extract_readytime(s):
  # for looping over cooking_directions column, which is a string formed like a dict
  time = re.findall(readyin_regex, ast.literal_eval(s)['directions'])
  if time: 
    hours_mins = time[0].split('\n')[1]

    try:
      days = re.search('(\d+) d', hours_mins).group(1)
    except:
      days = '0'
    
    try:
      hours = re.search('(\d+) h', hours_mins).group(1)
    except:
      hours = '0'
    
    try:
      minutes = re.search('(\d+) m', hours_mins).group(1)
    except:
      minutes = '0'
    
    total_minutes = int(days)*24*60 + int(hours)*60 + int(minutes)
    return total_minutes
  return None

# Make new column with total (ready in) time
raw_recipes_df['ready_time_minutes'] = raw_recipes_df['cooking_directions'].apply(extract_readytime)

# keep only the rows that have a 'ready-in' time
raw_recipes_df = raw_recipes_df.loc[raw_recipes_df['ready_time_minutes'] != 'None']

raw_recipes_df

,recipe_id,recipe_name,aver_rate,review_nums,ingredients,cooking_directions,ready_time_minutes
0,222388,Homemade Bacon,5.000000,3,pork belly^smoked paprika^kosher salt^ground b...,{'directions': u'Prep\n5 m\nCook\n2 h 45 m\nRe...,710.0
1,240488,"Pork Loin, Apples, and Sauerkraut",4.764706,29,sauerkraut drained^Granny Smith apples sliced^...,{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...,165.0
2,218939,Foolproof Rosemary Chicken Wings,4.571429,12,chicken wings^sprigs rosemary^head garlic^oliv...,"{'directions': u""Prep\n20 m\nCook\n40 m\nReady...",60.0
3,87211,Chicken Pesto Paninis,4.625000,163,focaccia bread quartered^prepared basil pesto^...,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,20.0
4,245714,Potato Bacon Pizza,4.500000,2,red potatoes^strips bacon^Sauce:^heavy whippin...,{'directions': u'Prep\n20 m\nCook\n45 m\nReady...,70.0
...,...,...,...,...,...,...,...
49693,222886,Grateful Dead Cocktail,3.500000,4,fluid ounce tequila^fluid ounce vodka^fluid ou...,{'directions': u'Prep\n5 m\nReady In\n5 m\nCom...,5.0
49694,25650,Cheese Filling For Pastries,4.333333,3,raisins^brandy^cream cheese^white sugar^all-pu...,{'directions': u'Prep\n15 m\nReady In\n15 m\nS...,15.0
49695,23544,Peach Smoothie,3.615385,21,sliced peaches drained^scoops vanilla ice crea...,{'directions': u'Prep\n1 m\nReady In\n1 m\nIn ...,1.0
49696,170710,Double Dare Peaches,4.714286,19,butter^habanero peppers^fresh peaches^brown su...,{'directions': u'Prep\n20 m\nCook\n10 m\nReady...,30.0


In [ ]:
s = raw_recipes_df.iloc[99]['cooking_directions']
s

"{'directions': u'Prep\\n25 m\\nCook\\n40 m\\nReady In\\n1 h 5 m\\nPreheat oven to 375 degrees F (190 degrees C).\\nIn a large heavy skillet, over medium-high heat, melt half of the butter. Fry the pork chops on both sides until a nice golden color. Transfer the chops to a 2 quart casserole dish with a lid, and sprinkle with thyme.\\nMelt the remaining butter in the pan, and fry the onion and garlic until tender. Transfer them to the casserole dish. Fry the apple rings in the pan for a few seconds on each side to give them color, then remove to the casserole dish. Sprinkle with sugar.\\nSpoon any excess fat from the frying pan and pour in the apple cider. Stir to remove any flavorful bits from the bottom. Bring the mixture to a simmer, then pour over the pork chops and apples in the dish. Season with salt and pepper.\\nBake the casserole, covered for 30 to 40 minutes, or until the pork chops are cooked through. Remove the pork chops and apples to a serving platter and stir the cream in

Drop rows where ready_time is NaN

In [ ]:
raw_recipes_df = raw_recipes_df[raw_recipes_df['ready_time_minutes'].notna()]
raw_recipes_df

,recipe_id,recipe_name,aver_rate,review_nums,ingredients,cooking_directions,ready_time_minutes
0,222388,Homemade Bacon,5.000000,3,pork belly^smoked paprika^kosher salt^ground b...,{'directions': u'Prep\n5 m\nCook\n2 h 45 m\nRe...,710.0
1,240488,"Pork Loin, Apples, and Sauerkraut",4.764706,29,sauerkraut drained^Granny Smith apples sliced^...,{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...,165.0
2,218939,Foolproof Rosemary Chicken Wings,4.571429,12,chicken wings^sprigs rosemary^head garlic^oliv...,"{'directions': u""Prep\n20 m\nCook\n40 m\nReady...",60.0
3,87211,Chicken Pesto Paninis,4.625000,163,focaccia bread quartered^prepared basil pesto^...,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,20.0
4,245714,Potato Bacon Pizza,4.500000,2,red potatoes^strips bacon^Sauce:^heavy whippin...,{'directions': u'Prep\n20 m\nCook\n45 m\nReady...,70.0
...,...,...,...,...,...,...,...
49693,222886,Grateful Dead Cocktail,3.500000,4,fluid ounce tequila^fluid ounce vodka^fluid ou...,{'directions': u'Prep\n5 m\nReady In\n5 m\nCom...,5.0
49694,25650,Cheese Filling For Pastries,4.333333,3,raisins^brandy^cream cheese^white sugar^all-pu...,{'directions': u'Prep\n15 m\nReady In\n15 m\nS...,15.0
49695,23544,Peach Smoothie,3.615385,21,sliced peaches drained^scoops vanilla ice crea...,{'directions': u'Prep\n1 m\nReady In\n1 m\nIn ...,1.0
49696,170710,Double Dare Peaches,4.714286,19,butter^habanero peppers^fresh peaches^brown su...,{'directions': u'Prep\n20 m\nCook\n10 m\nReady...,30.0


# Extract Recipe Text, minus the time headers. Inspect for other anomalies to filter out.

In [ ]:
s = raw_recipes_df.iloc[0]['cooking_directions']
print(s)

{'directions': u'Prep\n5 m\nCook\n2 h 45 m\nReady In\n11 h 50 m\nPreheat oven to 200 degrees F (95 degrees C).\nSeason pork belly with paprika, salt, and pepper. Tightly wrap pork twice in heavy-duty aluminum foil. Place on a baking sheet and bake in the preheated oven for 2 1/2 hours. Turn off the oven; let pork rest in the oven for 1 hour. Remove meat from oven, leaving it wrapped in aluminum foil, and refrigerate at least 8 hours or overnight.\nRemove pork from foil and slice across the grain in 1/4-inch thick slices. Working in batches, cook pork in a non-stick skillet over medium heat until golden and crisped, 6 to 8 minutes per slice.'}


In [ ]:
# remove the text about time, like: "Prep\n5 m\nCook\n2 h 45 m\nReady In\n11 h 50 m\n"
def remove_time_headers(recipe_text):
  try:
    p1 = recipe_text.split('Ready In\\n')[1]
    p2 = re.match('.+?\\\\n(.*)', p1).group(1)
    return p2
  except:
    return 'ERROR'
  
remove_time_headers( raw_recipes_df['cooking_directions'].iloc[99] )

"Heat olive oil and 1 tablespoon butter in a large Dutch oven over medium heat. Add pancetta; cook and stir until browned, about 8 minutes. Add celery, carrots, and onion; cook, stirring occasionally, until very soft, about 15 minutes. Stir in beef and pork. Season with salt and pepper. Cook until evenly browned, about 20 minutes.\\nPour white wine into the Dutch oven; cook until alcohol evaporates, about 5 minutes. Stir in crushed tomatoes, tomato paste, and sugar. Bring to a simmer; reduce heat to low, cover and simmer Bolognese sauce, stirring occasionally and skimming fat off the surface, about 3 1/2 hours. Stir in beef broth, 1/4 cup at a time, if sauce looks dry.\\nStir heavy cream and 1/2 cup milk into the Bolognese sauce; mix well. Cover and cook over very low heat until flavors combine, about 30 minutes.\\nMelt 5 tablespoons butter in a large saucepan over medium heat. Stir in flour until a smooth paste forms; cook and stir until paste turns a golden sandy color, about 7 minut

In [ ]:
raw_recipes_df['directions_text'] = raw_recipes_df['cooking_directions'].apply(remove_time_headers)
raw_recipes_df

<ipython-input-10-6c5fd288f0e4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_recipes_df['directions_text'] = raw_recipes_df['cooking_directions'].apply(remove_time_headers)


,recipe_id,recipe_name,aver_rate,review_nums,ingredients,cooking_directions,ready_time_minutes,directions_text
0,222388,Homemade Bacon,5.000000,3,pork belly^smoked paprika^kosher salt^ground b...,{'directions': u'Prep\n5 m\nCook\n2 h 45 m\nRe...,710.0,Preheat oven to 200 degrees F (95 degrees C).\...
1,240488,"Pork Loin, Apples, and Sauerkraut",4.764706,29,sauerkraut drained^Granny Smith apples sliced^...,{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...,165.0,Preheat oven to 325 degrees F (165 degrees C)....
2,218939,Foolproof Rosemary Chicken Wings,4.571429,12,chicken wings^sprigs rosemary^head garlic^oliv...,"{'directions': u""Prep\n20 m\nCook\n40 m\nReady...",60.0,Preheat an oven to 350 degrees F (175 degrees ...
3,87211,Chicken Pesto Paninis,4.625000,163,focaccia bread quartered^prepared basil pesto^...,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,20.0,Preheat a panini grill.\nSlice each quarter of...
4,245714,Potato Bacon Pizza,4.500000,2,red potatoes^strips bacon^Sauce:^heavy whippin...,{'directions': u'Prep\n20 m\nCook\n45 m\nReady...,70.0,Place potato slices in a deep skillet and cove...
...,...,...,...,...,...,...,...,...
49693,222886,Grateful Dead Cocktail,3.500000,4,fluid ounce tequila^fluid ounce vodka^fluid ou...,{'directions': u'Prep\n5 m\nReady In\n5 m\nCom...,5.0,"Combine tequila, vodka, rum, gin, and raspberr..."
49694,25650,Cheese Filling For Pastries,4.333333,3,raisins^brandy^cream cheese^white sugar^all-pu...,{'directions': u'Prep\n15 m\nReady In\n15 m\nS...,15.0,Sprinkle raisins with brandy and set aside.\nI...
49695,23544,Peach Smoothie,3.615385,21,sliced peaches drained^scoops vanilla ice crea...,{'directions': u'Prep\n1 m\nReady In\n1 m\nIn ...,1.0,"In a blender, combine peaches, ice cream, soy ..."
49696,170710,Double Dare Peaches,4.714286,19,butter^habanero peppers^fresh peaches^brown su...,{'directions': u'Prep\n20 m\nCook\n10 m\nReady...,30.0,Melt the butter in a large skillet over medium...


Replace newlines in recipe with spaces; export as new csv file

In [ ]:
def remove_newlines(s):
  return s.replace('\\n', ' ')
raw_recipes_df['directions_text'] = raw_recipes_df['directions_text'].apply(remove_newlines)
raw_recipes_df

<ipython-input-11-056107976245>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_recipes_df['directions_text'] = raw_recipes_df['directions_text'].apply(remove_newlines)


,recipe_id,recipe_name,aver_rate,review_nums,ingredients,cooking_directions,ready_time_minutes,directions_text
0,222388,Homemade Bacon,5.000000,3,pork belly^smoked paprika^kosher salt^ground b...,{'directions': u'Prep\n5 m\nCook\n2 h 45 m\nRe...,710.0,Preheat oven to 200 degrees F (95 degrees C). ...
1,240488,"Pork Loin, Apples, and Sauerkraut",4.764706,29,sauerkraut drained^Granny Smith apples sliced^...,{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...,165.0,Preheat oven to 325 degrees F (165 degrees C)....
2,218939,Foolproof Rosemary Chicken Wings,4.571429,12,chicken wings^sprigs rosemary^head garlic^oliv...,"{'directions': u""Prep\n20 m\nCook\n40 m\nReady...",60.0,Preheat an oven to 350 degrees F (175 degrees ...
3,87211,Chicken Pesto Paninis,4.625000,163,focaccia bread quartered^prepared basil pesto^...,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,20.0,Preheat a panini grill. Slice each quarter of ...
4,245714,Potato Bacon Pizza,4.500000,2,red potatoes^strips bacon^Sauce:^heavy whippin...,{'directions': u'Prep\n20 m\nCook\n45 m\nReady...,70.0,Place potato slices in a deep skillet and cove...
...,...,...,...,...,...,...,...,...
49693,222886,Grateful Dead Cocktail,3.500000,4,fluid ounce tequila^fluid ounce vodka^fluid ou...,{'directions': u'Prep\n5 m\nReady In\n5 m\nCom...,5.0,"Combine tequila, vodka, rum, gin, and raspberr..."
49694,25650,Cheese Filling For Pastries,4.333333,3,raisins^brandy^cream cheese^white sugar^all-pu...,{'directions': u'Prep\n15 m\nReady In\n15 m\nS...,15.0,Sprinkle raisins with brandy and set aside. In...
49695,23544,Peach Smoothie,3.615385,21,sliced peaches drained^scoops vanilla ice crea...,{'directions': u'Prep\n1 m\nReady In\n1 m\nIn ...,1.0,"In a blender, combine peaches, ice cream, soy ..."
49696,170710,Double Dare Peaches,4.714286,19,butter^habanero peppers^fresh peaches^brown su...,{'directions': u'Prep\n20 m\nCook\n10 m\nReady...,30.0,Melt the butter in a large skillet over medium...


In [ ]:
# TODO: make a few extra features - maybe length of directions, and number of different ingredients
raw_recipes_df['direction_text_num_words'] = raw_recipes_df['directions_text'].apply(lambda x: len(x.split(' ')))
raw_recipes_df['num_ingredients'] = raw_recipes_df['ingredients'].apply(lambda x: len(x.split('^')))
raw_recipes_df.head()

<ipython-input-15-df977a535b38>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_recipes_df['direction_text_num_words'] = raw_recipes_df['directions_text'].apply(lambda x: len(x.split(' ')))
<ipython-input-15-df977a535b38>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_recipes_df['num_ingredients'] = raw_recipes_df['ingredients'].apply(lambda x: len(x.split('^')))


,recipe_id,recipe_name,aver_rate,review_nums,ingredients,cooking_directions,ready_time_minutes,directions_text,direction_text_num_words,num_ingredients
0,222388,Homemade Bacon,5.000000,3,pork belly^smoked paprika^kosher salt^ground b...,{'directions': u'Prep\n5 m\nCook\n2 h 45 m\nRe...,710.0,Preheat oven to 200 degrees F (95 degrees C). ...,106,4
1,240488,"Pork Loin, Apples, and Sauerkraut",4.764706,29,sauerkraut drained^Granny Smith apples sliced^...,{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...,165.0,Preheat oven to 325 degrees F (165 degrees C)....,131,12
2,218939,Foolproof Rosemary Chicken Wings,4.571429,12,chicken wings^sprigs rosemary^head garlic^oliv...,"{'directions': u""Prep\n20 m\nCook\n40 m\nReady...",60.0,Preheat an oven to 350 degrees F (175 degrees ...,155,6
3,87211,Chicken Pesto Paninis,4.625000,163,focaccia bread quartered^prepared basil pesto^...,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,20.0,Preheat a panini grill. Slice each quarter of ...,57,6
4,245714,Potato Bacon Pizza,4.500000,2,red potatoes^strips bacon^Sauce:^heavy whippin...,{'directions': u'Prep\n20 m\nCook\n45 m\nReady...,70.0,Place potato slices in a deep skillet and cove...,255,14


In [ ]:
raw_recipes_df.to_csv(datapath+'clean_recipe_data.csv')

In [ ]:
# TODO: make a file for openrefine, so you can cluster ingredients? nah... though, you'd just make a 2 by X table where
# the X rows each have an ingredient on them, and the second column is a duplicate of the first, so when you
# cluster the values in column 2 you end up with a map from original (col 1) to clustered (col 2)

# This is probably not useful at the moment (ex. what would you even do with 10,000 features? maybe the text is important tho)